In [1]:
import yaml
import torch
import random

from torch.utils.data import Subset

from model.model import Model
from model.trainer import Trainer
from model.test import Test
from model.model_analyze import ModelAnalyze
from data_pre.dataset import TrainValidDataset, TestDataset

random_seed = 42
random.seed(random_seed)

# Model & Tokenizer & Dataset 생성

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

with open('/root/upstage-nlp-summarization-nlp2/config.yaml', "r") as file:
    config = yaml.safe_load(file)

# Model & Tokenizer
model_instance = Model(config, device)
model = model_instance.getModel()
tokenizer = model_instance.getTokenizer()

# dataset
train_dataset = TrainValidDataset(config, tokenizer, is_train=True)
val_dataset = TrainValidDataset(config, tokenizer, is_train=False)
test_dataset = TestDataset(config, tokenizer)

/opt/conda/envs/myvenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


# Trainer 학습

In [5]:
trainer = Trainer(config, model, train_dataset, val_dataset, tokenizer).get_trainer()
trainer.train()

/opt/conda/envs/myvenv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/envs/myvenv/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,No log,1.614725,0.143275,0.027206,0.139412
2,4.245200,0.608342,0.361342,0.123474,0.346808
3,0.389600,0.580516,0.364083,0.128525,0.350191
4,0.325400,0.565599,0.370407,0.132611,0.354429
5,0.325400,0.560738,0.376468,0.139522,0.360432
6,0.295900,0.558430,0.379644,0.139568,0.363962
7,0.276600,0.560502,0.381270,0.144705,0.365656
8,0.259700,0.562498,0.374397,0.134737,0.359304
9,0.247200,0.565898,0.374683,0.138324,0.360761
10,0.247200,0.569009,0.379212,0.141993,0.363684


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-defau

TrainOutput(global_step=4290, training_loss=0.746473709853379, metrics={'train_runtime': 1071.7407, 'train_samples_per_second': 232.463, 'train_steps_per_second': 7.278, 'total_flos': 4.177517883162624e+16, 'train_loss': 0.746473709853379, 'epoch': 11.0})

# Test 추론

In [3]:
model_path = '/root/upstage-nlp-summarization-nlp2/results/checkpoint-2340'
test_inst = Test(config, test_dataset, tokenizer, model_path, device)
result_df = test_inst()
result_df

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
100%|██████████| 16/16 [00:40<00:00,  2.53s/it]


,fname,summary
0,test_0,#Person1#은 더슨 씨에게 사무실 통신이 이메일 통신과 공식 메모로 제한된다고...
1,test_1,#Person2#는 교통 체증에 걸렸다. #Person1#는 #Person2#가 대...
2,test_2,케이트는 마샤와 히어로가 이혼하려고 한다고 #Person1#에게 말한다. #Pers...
3,test_3,#Person1#은 브라이언의 생일을 축하하기 위해 파티에 왔다. #Person2#...
4,test_4,#Person2#는 #Person1#에게 올림픽 스타디움에 5000개의 좌석이 있다...
...,...,...
494,test_495,찰리는 아빠를 데리러 가야하기 때문에 #Person1# 과 함께 비디오 게임을 하고...
495,test_496,#Person2#는 컨트리 음악에 관심을 가지게 된 계기와 라디오 방송국에서 일하기...
496,test_497,"앨리스는 #Person1#에게 기계를 어떻게 사용하는지 알려주고, 기계가 너무 많은..."
497,test_498,스티브가 매튜에게 전화를 걸어 계약이 다음 달에 끝나기 때문에 집을 찾고 있다고 말...


# Model 성능 분석

In [4]:
# train_dataset의 0.2퍼 가져오기

sample_size = int(len(train_dataset) * 0.2)
indices = random.sample(range(len(train_dataset)), sample_size)

valid_dataset = Subset(train_dataset, indices)
model_path = '/root/upstage-nlp-summarization-nlp2/results/checkpoint-2340'

In [5]:
model_analyze = ModelAnalyze(config, tokenizer, device)
result_df = model_analyze.get_result(valid_dataset, model_path)
result_df

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
  0%|          | 0/78 [00:00<?, ?it/s]


TypeError: argument 'ids': 'list' object cannot be interpreted as an integer